In [1]:
#!/usr/bin/env python3
"""
 ==============================================================================
 SMB WAVE PREDICTION CALCULATOR
 ==============================================================================
 MODULE:      calculator.ipynb
 TYPE:        Empirical Wave Prediction Tool
 METHOD:      Revised Sverdrup-Munk-Bretschneider (SMB)
 STANDARDS:   U.S. Army Coastal Engineering Manual (CEM)
              Shore Protection Manual (SPM 1984)
              Hurdle & Stive (1989) Unified Formulations
 LICENSE:     MIT / Academic Open Source
 ==============================================================================

 ------------------------------------------------------------------------------
 1. THEORETICAL MANUAL & PHYSICS ENGINE DOCUMENTATION
 ------------------------------------------------------------------------------

 1.1 PHYSICAL OVERVIEW
 This software predicts the characteristics of wind-generated waves (Significant
 Wave Height Hs and Period Ts) based on the Sverdrup-Munk-Bretschneider (SMB)
 method.

 1.2 ATMOSPHERIC FORCING (WIND STRESS)
 The driving force for wave generation is the wind stress on the water surface.
 Standard meteorological wind speed is measured at 10m elevation (U10).
 However, the empirical curves in the SPM (1984) were calibrated using an
 "Adjusted Wind Speed" (Ua), also known as the wind stress factor.

 FORMULA: Ua = 0.71 * (U10)^1.23
 - U10: Wind speed at 10m elevation [m/s]
 - Ua:  Adjusted wind speed [m/s]

 1.3 DIMENSIONAL ANALYSIS & PARAMETERS
 The governing physics are described by the following dimensionless groups:

 - Dimensionless Fetch (F_hat):    F_hat = (g * F) / Ua^2
 - Dimensionless Depth (d_hat):    d_hat = (g * d) / Ua^2
 - Dimensionless Duration (t_hat): t_hat = (g * t) / Ua

 Where:
 - g = Gravitational acceleration (9.8066 m/s^2)
 - F = Fetch Length [m]
 - d = Water Depth [m]
 - t = Duration [s]

 1.4 WAVE GROWTH EQUATIONS (FETCH-LIMITED - Hurdle & Stive, 1989)
 The revised equations provide a single unified formulation for both deep and
 finite-depth water, avoiding inconsistencies found in the original SPM.

 A. Significant Wave Height (Hs):
 (g * Hs) / Ua^2 = 0.25 * tanh[K_d1] * tanh^0.5 [ (4.3e-5 * F_hat) / tanh^2(K_d1) ]
 Where K_d1 = 0.6 * (d_hat)^0.75

 B. Significant Wave Period (Ts):
 (g * Ts) / Ua   = 8.3 * tanh[K_d2] * tanh^1/3 [ (4.1e-5 * F_hat) / tanh^3(K_d2) ]
 Where K_d2 = 0.76 * (d_hat)^0.375

 Note: In Deep Water (d_hat -> inf), the depth terms tanh(K_d) approach 1.0.

 1.5 WAVE GROWTH EQUATIONS (DURATION-LIMITED)
 When the storm duration (t) is the limiting factor, the software utilizes the
 "Effective Fetch" method (Hurdle & Stive, 1989, Eq 4.13) to ensure kinematic
 consistency with the fetch-limited curves.

 Algorithm:
 1. Calculate Dimensionless Duration: t_hat = (g * t) / Ua
 2. Invert the minimum duration power law to find Effective Fetch (F_eff):
 F_hat_eff = (t_hat / 65.9)^(1.5)
 3. Substitute F_hat_eff into the standard Fetch-Limited equations (1.4) to
 solve for Hs and Ts.

 1.6 EXACT WAVE KINEMATICS (DISPERSION RELATION)
 To convert the Period (Ts) into Wavelength (L), we solve the Linear Dispersion
 Relation for water waves. This is a transcendental equation that cannot be
 solved analytically for intermediate depths.

 Equation: L = (g * T^2 / 2pi) * tanh( 2pi * d / L )

 Numerical Solution (Newton-Raphson):
 We solve for the dimensionless wavenumber 'kh' = k * d.
 Function: f(kh) = (w^2 * d / g) - kh * tanh(kh) = 0
 Derivative: f'(kh) = -tanh(kh) - kh * sech^2(kh)

 1.7 STABILITY CRITERIA (BREAKING)
 The model checks if the predicted waves are physically sustainable.

 A. Miche Criterion (Steepness Breaking):
 Waves break when the particle velocity at the crest exceeds the phase speed.
 Limit: H / L <= 0.142 * tanh(k * d)
 In deep water, this simplifies to H/L <= 1/7.
"""

import math
import sys

In [2]:
# ==============================================================================
#  SECTION 1: CORE TYPES & CONSTANTS
# ==============================================================================

class Phys:
    G_STD = 9.8066  # Standard Gravity [m/s^2]
    PI    = 3.14159265358979323846

In [3]:
# ==============================================================================
#  SECTION 2: LOGGING UTILITIES
#  Handles the formatting of the text report displayed in the output window.
# ==============================================================================

class StringLogger:
    def __init__(self):
        self.buffer = []
        self.width_label = 25
        self.width_val   = 12
        self.width_unit  = 8
        self.line_len    = 60

    # Prints a major section header with double borders
    def print_header(self, title):
        self.buffer.append('=' * self.line_len)
        self.buffer.append(f"  {title}")
        self.buffer.append('=' * self.line_len)

    # Prints a sub-section header with single borders
    def print_subheader(self, title):
        self.buffer.append("")
        self.buffer.append('-' * self.line_len)
        self.buffer.append(f"  {title}")
        self.buffer.append('-' * self.line_len)
    
    # Prints a standard data row: "Label : Value Unit"
    def print_row(self, desc, val, unit):
        if abs(val) < 1e-9:
            s_val = "0.00"
        else:
            s_val = f"{val:.4f}"
        
        # String formatting
        row = f"  {desc:<{self.width_label}} : {s_val:<{self.width_val}}{unit:<{self.width_unit}}"
        self.buffer.append(row)

    # Prints a string data row: "Label : Value"
    def print_str(self, desc, val, unit=""):
        row = f"  {desc:<{self.width_label}} : {val:<{self.width_val}}{unit:<{self.width_unit}}"
        self.buffer.append(row)

    # Prints a plain text line (indented)
    def print_text(self, text):
        self.buffer.append(f"  {text}")

    # Inserts a blank line
    def newline(self):
        self.buffer.append("")

    # Returns the complete report string
    def get_content(self):
        return "\n".join(self.buffer)

In [4]:
# ==============================================================================
#  SECTION 3: PHYSICS ENGINE (SMB & DISPERSION)
# ==============================================================================

class SMBResult:
    def __init__(self, Hs=0.0, Ts=0.0, t_min=0.0, equiv_fetch=0.0):
        self.Hs = Hs              # Significant Wave Height [m]
        self.Ts = Ts              # Significant Wave Period [s]
        self.t_min = t_min        # Minimum Duration required for fetch-limited state [Hours]
        self.equiv_fetch = equiv_fetch # Equivalent Fetch length [km]

class SMBEngine:
    # --- 3.1 Empirical Formulas (Revised SMB - Hurdle & Stive, 1989) ---

    # Calculates Adjusted Wind Speed (Ua).
    # Correction for the non-linear relationship between wind speed and stress.
    # Formula: Ua = 0.71 * U10^1.23
    @staticmethod
    def calculate_adjusted_wind_speed(U10):
        return 0.71 * math.pow(U10, 1.23)

    # Calculates Fetch-Limited parameters for Deep Water.
    # Uses the asymptotic limits of the Hurdle & Stive (1989) unified equations.
    @staticmethod
    def calculate_deep_water(Ua, fetch_m):
        # Dimensionless Fetch: F_hat = g * F / Ua^2
        dim_fetch = (Phys.G_STD * fetch_m) / (Ua * Ua)

        # Revised Significant Wave Height (Hs) - Deep Water Asymptote
        # Formula: (g * Hs) / Ua^2 = 0.25 * [ tanh(4.3e-5 * F_hat) ]^0.5
        term_fetch_h = 4.3e-5 * dim_fetch
        gHs_U2 = 0.25 * math.pow(math.tanh(term_fetch_h), 0.5)
        Hs = gHs_U2 * (Ua * Ua / Phys.G_STD)

        # Revised Significant Wave Period (Ts) - Deep Water Asymptote
        # Formula: (g * Ts) / Ua = 8.3 * [ tanh(4.1e-5 * F_hat) ]^(1/3)
        term_fetch_t = 4.1e-5 * dim_fetch
        gTs_U = 8.3 * math.pow(math.tanh(term_fetch_t), 1.0/3.0)
        Ts = gTs_U * (Ua / Phys.G_STD)

        # Minimum Duration (t_min) - REVISED (Hurdle & Stive, 1989, Eq 4.12)
        # The paper explicitly recommends using this simple power law in all cases
        # to maintain consistency with the effective fetch inversion.
        # Formula: t_hat = 65.9 * F_hat^(2/3)
        dim_duration = 65.9 * math.pow(dim_fetch, 2.0/3.0)
        t_min_hr = (dim_duration * Ua / Phys.G_STD) / 3600.0

        return SMBResult(Hs, Ts, t_min_hr, 0.0)

    # Calculates Fetch-Limited parameters for Finite Depth.
    # Uses the unified equations from Hurdle & Stive (1989).
    @staticmethod
    def calculate_depth_limited(Ua, fetch_m, depth):
        dim_fetch = (Phys.G_STD * fetch_m) / (Ua * Ua)
        dim_depth = (Phys.G_STD * depth) / (Ua * Ua)

        # Revised Significant Wave Height (Hs) - Hurdle & Stive (1989) Eq 4.1
        # Depth-damping: K_d1 = tanh(0.6 * d_hat^0.75)
        depth_term_h = math.tanh(0.6 * math.pow(dim_depth, 0.75))
        # Protection against division by zero if depth is extremely small
        if depth_term_h < 1e-6: 
            depth_term_h = 1e-6

        fetch_term_inner_h = (4.3e-5 * dim_fetch) / (depth_term_h * depth_term_h)
        
        gHs_U2 = 0.25 * depth_term_h * math.pow(math.tanh(fetch_term_inner_h), 0.5)
        Hs = gHs_U2 * (Ua * Ua / Phys.G_STD)

        # Revised Significant Wave Period (Ts) - Hurdle & Stive (1989) Eq 4.2
        # Depth-damping: K_d2 = tanh(0.76 * d_hat^0.375)
        depth_term_t = math.tanh(0.76 * math.pow(dim_depth, 0.375))
        if depth_term_t < 1e-6: 
            depth_term_t = 1e-6

        fetch_term_inner_t = (4.1e-5 * dim_fetch) / math.pow(depth_term_t, 3.0)
        
        gTs_U = 8.3 * depth_term_t * math.pow(math.tanh(fetch_term_inner_t), 1.0/3.0)
        Ts = gTs_U * (Ua / Phys.G_STD)

        # Minimum Duration (t_min) - REVISED (Hurdle & Stive, 1989, Eq 4.12)
        # Ref: PDF Section 4.4 recommends using the deep water expression (Eq 3.15/4.12)
        # for limiting duration in all circumstances.
        dim_duration = 65.9 * math.pow(dim_fetch, 2.0/3.0)
        t_min_hr = (dim_duration * Ua / Phys.G_STD) / 3600.0

        return SMBResult(Hs, Ts, t_min_hr, 0.0)

    # Calculates Duration-Limited parameters.
    # Uses the "Effective Fetch" method (Hurdle & Stive, 1989, Eq 4.13).
    @staticmethod
    def calculate_duration_limited(Ua, dur_hr, depth, is_deep):
        # 1. Calculate Dimensionless Duration
        dur_sec = dur_hr * 3600.0
        dim_dur = (Phys.G_STD * dur_sec) / Ua

        # 2. Calculate Effective Fetch (Eq 4.13)
        # Formula: F_hat' = (t_hat / 65.9)^(3/2)
        dim_fetch_eq = math.pow(dim_dur / 65.9, 1.5)
        
        # Convert back to meters
        equiv_fetch_m = dim_fetch_eq * (Ua * Ua / Phys.G_STD)

        # 3. Calculate Waves using Effective Fetch
        if is_deep:
            res = SMBEngine.calculate_deep_water(Ua, equiv_fetch_m)
        else:
            res = SMBEngine.calculate_depth_limited(Ua, equiv_fetch_m, depth)

        # Update the equivalent fetch field (convert to km) for reporting
        res.equiv_fetch = equiv_fetch_m / 1000.0
        
        # Ensure t_min reflects the input duration for this logic branch
        res.t_min = dur_hr

        return res

    # --- 3.2 Exact Dispersion Solver (Newton-Raphson) ---
    # Solves the transcendental Linear Dispersion Relation.
    # Equation: L = (g * T^2 / 2pi) * tanh( 2pi * d / L )
    @staticmethod
    def solve_dispersion(T, d, tol=1e-15, max_iter=100):
        if d <= 0: return 0.0
        omega = 2.0 * Phys.PI / T
        k0 = (omega * omega) / Phys.G_STD
        k0h = k0 * d

        # Initial Guess using Carvalho (2006) explicit approximation
        kh = k0h
        if k0h > 0:
             kh = k0h / math.tanh(math.pow(6.0/5.0, k0h) * math.sqrt(k0h))
        else:
             return 0.0

        # Newton-Raphson Iteration loop
        for i in range(max_iter):
            t_kh = math.tanh(kh)
            f = k0h - kh * t_kh
            sech = 1.0 / math.cosh(kh)
            df = -t_kh - kh * (sech * sech)
            
            if abs(df) < 1e-15: 
                break
            
            dkh = f / df
            kh_new = kh - dkh
            
            # Convergence check
            if abs(dkh / kh) < tol: 
                return kh_new
            kh = kh_new
        
        return kh

In [5]:
# ==============================================================================
#  SECTION 4: SIMULATION LOGIC
# ==============================================================================

class AppInputs:
    def __init__(self, U10, fetch_km, duration_hr, depth_m):
        self.U10 = U10
        self.fetch_km = fetch_km
        self.duration_hr = duration_hr # -1 if infinite
        self.depth_m = depth_m         # -1 if deep water

def RunSimulation(inp):
    log = StringLogger()
    log.print_header("SMB WAVE PREDICTION REPORT")
    log.print_text("Methodology: Revised SMB")
    log.print_text("Ref: Hurdle & Stive (1989) Unified Formulations")

    # --- 1. Physics Calculations (SMB) ---
    Ua = SMBEngine.calculate_adjusted_wind_speed(inp.U10)
    fetch_m = inp.fetch_km * 1000.0
    deep_water = (inp.depth_m < 0)
    inf_duration = (inp.duration_hr < 0)

    # Calculate Potential A: Fetch Limited
    # (Assumes infinite duration, limited only by distance)
    if deep_water:
        r_fetch = SMBEngine.calculate_deep_water(Ua, fetch_m)
    else:
        r_fetch = SMBEngine.calculate_depth_limited(Ua, fetch_m, inp.depth_m)

    # Calculate Potential B: Duration Limited
    # (Assumes infinite fetch, limited only by time)
    dur_val = 1e9 if inf_duration else inp.duration_hr
    r_dur = SMBEngine.calculate_duration_limited(Ua, dur_val, inp.depth_m, deep_water)

    # Controlling Factor Logic
    # The actual sea state is the smaller of the two potentials (Fetch vs Duration).
    if r_fetch.Hs <= r_dur.Hs:
        final_Hs = r_fetch.Hs
        final_Ts = r_fetch.Ts
        control_msg = "FETCH-LIMITED"
        rel_dur = r_fetch.t_min
        dur_label = "Min Duration Req."
        rel_fetch = inp.fetch_km
        fetch_label = "Given Fetch"
        r_selected = r_fetch
    else:
        final_Hs = r_dur.Hs
        final_Ts = r_dur.Ts
        control_msg = "DURATION-LIMITED"
        rel_dur = inp.duration_hr
        dur_label = "Given Duration"
        rel_fetch = r_dur.equiv_fetch
        fetch_label = "Equivalent Fetch"
        r_selected = r_dur

    # --- 2. Precise Hydrodynamics (Dispersion) ---
    # Real k, L, C, kh, d_actual;

    if deep_water:
        # Deep water dispersion (Analytic): L0 = gT^2 / 2pi
        L = (Phys.G_STD * final_Ts * final_Ts) / (2 * Phys.PI)
        k = 2 * Phys.PI / L
        kh = 100.0 # Effectively infinite
        d_actual = 1e9 # Large number for display logic
    else:
        d_actual = inp.depth_m
        # Finite depth dispersion (Numerical): Solve Newton-Raphson
        kh = SMBEngine.solve_dispersion(final_Ts, d_actual)
        k = kh / d_actual
        L = 2 * Phys.PI / k
    
    C = L / final_Ts # Phase Speed

    # --- 3. Report Generation ---

    # INPUTS
    log.print_subheader("1. INPUT PARAMETERS")
    log.print_row("Wind Speed (U10)", inp.U10, "m/s")
    log.print_row("Adjusted Speed (Ua)", Ua, "m/s")
    log.print_row("Fetch Length (F)", inp.fetch_km, "km")
    
    if inf_duration:
        log.print_str("Storm Duration (t_act)", "Infinite", "hours")
    else:
        log.print_row("Storm Duration (t_act)", inp.duration_hr, "hours")

    if deep_water:
        log.print_str("Water Depth (d)", "Deep Water", "-")
    else:
        log.print_row("Water Depth (d)", inp.depth_m, "m")

    # CONDITIONS
    log.print_subheader("2. LIMITING CONDITIONS")
    
    # Controlling factor
    log.print_str(">> Controlling Factor", control_msg)
    
    log.print_text(">> Determining Parameters:")
    
    # Indented parameters
    if inf_duration and control_msg == "FETCH-LIMITED":
        log.print_row("   " + dur_label, rel_dur, "hours")
    elif inf_duration:
         log.print_str("   " + dur_label, "Infinite", "hours")
    else:
        log.print_row("   " + dur_label, rel_dur, "hours")
    
    log.print_row("   " + fetch_label, rel_fetch, "km")

    # RESULTS
    log.print_subheader("3. PREDICTION RESULTS")
    log.print_row("Sig. Wave Height (Hs)", final_Hs, "m")
    log.print_row("Sig. Wave Period (Ts)", final_Ts, "s")
    log.print_row("Wave Celerity (C)", C, "m/s")
    log.print_row("Wave Length (L)", L, "m")
    log.print_row("Wave Number (k)", k, "rad/m")

    # INSIGHTS
    log.print_subheader("4. INSIGHTS & OBSERVATIONS")
    
    # Insight A: Growth State
    if control_msg == "FETCH-LIMITED":
        log.print_text(">> State: Fully Developed for Fetch.")
        log.print_text("   Waves have reached maximum size for this distance.")
    else:
        log.print_text(">> State: Growing Sea State (Duration Limited).")
        log.print_text("   Waves are still growing over time.")

    # Insight B: Regime & Stability
    regime = ""
    d_L = 1.0 if deep_water else (d_actual / L)
    
    if d_L >= 0.5: 
        regime = "Deep Water (d/L > 0.5)"
    elif d_L < 0.05: 
        regime = "Shallow Water (d/L < 0.05)"
    else: 
        regime = "Transitional/Intermediate"
    
    log.print_str(">> Flow Regime", regime)
    
    # Breaking (Miche Criterion)
    # Limits wave steepness (H/L) in any water depth.
    # H_max / L = 0.142 * tanh(kd)
    steepness = final_Hs / L
    limit_steepness = 0.142 * math.tanh(kh)
    
    log.print_row("   Calculated Steepness", steepness, "-")
    log.print_row("   Miche Limit (H/L)", limit_steepness, "-")
    
    if steepness > limit_steepness:
        log.print_str(">> Breaking Status", "UNSTABLE / BREAKING [!]", "")
        log.print_text("   Wave height exceeds the Miche stability limit.")
    else:
        log.print_str(">> Breaking Status", "STABLE", "")
        safety_margin = (1.0 - steepness/limit_steepness) * 100.0
        log.print_row("   Stability Margin", safety_margin, "%")

    if not deep_water:
        ratio = final_Hs / d_actual
        log.print_row("   Depth Ratio (Hs/d)", ratio, "-")

    # SCENARIOS
    log.print_subheader("5. SCENARIO ANALYSIS")
    
    log.print_text("[A] Fetch-Limited Potential")
    log.print_row("    - Potential Hs", r_fetch.Hs, "m")
    log.print_row("    - Potential Ts", r_fetch.Ts, "s")
    log.print_row("    - Time to Develop", r_fetch.t_min, "hours")
    
    log.newline()
    log.print_text("[B] Duration-Limited Potential")
    if inf_duration:
        log.print_text("    - Not applicable (Duration is infinite)")
    else:
        log.print_row("    - Potential Hs", r_dur.Hs, "m")
        log.print_row("    - Potential Ts", r_dur.Ts, "s")
        log.print_row("    - Equiv. Fetch", r_dur.equiv_fetch, "km")

    return log.get_content()

In [6]:
# ==============================================================================
#  SECTION 5: EXECUTION ENTRY POINT
# ==============================================================================

if __name__ == "__main__":
    print("========================================")
    print(" SMB WAVE PREDICTION INPUT")
    print(" Tip: Press ENTER to accept the Lake Garda defaults.")
    print("========================================\n")

    def get_input(label, default):
        """Prompts user for input, returns default if empty."""
        val = input(f"{label} [Default: {default}]: ").strip()
        return float(val) if val else default

    try:
        # 1. Wind Speed (Lake Garda Default: 25 m/s)
        u10 = get_input("Wind Speed (U10) [m/s]", 25.0)

        # 2. Fetch Length (Lake Garda Default: 45 km)
        fetch = get_input("Fetch Length [km]", 45.0)

        # 3. Duration (Lake Garda Default: Infinite)
        # We handle this manually to allow the user to type nothing for -1
        dur_input = input("Storm Duration [hours] [Default: Infinite]: ").strip()
        if not dur_input or dur_input.lower() == "infinite":
            duration = -1
        else:
            duration = float(dur_input)

        # 4. Water Depth (Lake Garda Default: 10 m)
        depth = get_input("Water Depth [m]", 10.0)

        # Create the inputs object
        inputs = AppInputs(U10=u10, fetch_km=fetch, duration_hr=duration, depth_m=depth)
        
        print("\nGenerating Report...\n")
        report_output = RunSimulation(inputs)
        print(report_output)

    except ValueError:
        print("\n[!] Error: Please enter valid numeric values.")

 SMB WAVE PREDICTION INPUT
 Tip: Press ENTER to accept the Lake Garda defaults.



Wind Speed (U10) [m/s] [Default: 25.0]:  
Fetch Length [km] [Default: 45.0]:  
Storm Duration [hours] [Default: Infinite]:  
Water Depth [m] [Default: 10.0]:  



Generating Report...

  SMB WAVE PREDICTION REPORT
  Methodology: Revised SMB
  Ref: Hurdle & Stive (1989) Unified Formulations

------------------------------------------------------------
  1. INPUT PARAMETERS
------------------------------------------------------------
  Wind Speed (U10)          : 25.0000     m/s     
  Adjusted Speed (Ua)       : 37.2156     m/s     
  Fetch Length (F)          : 45.0000     km      
  Storm Duration (t_act)    : Infinite    hours   
  Water Depth (d)           : 10.0000     m       

------------------------------------------------------------
  2. LIMITING CONDITIONS
------------------------------------------------------------
  >> Controlling Factor     : FETCH-LIMITED        
  >> Determining Parameters:
     Min Duration Req.      : 3.2407      hours   
     Given Fetch            : 45.0000     km      

------------------------------------------------------------
  3. PREDICTION RESULTS
------------------------------------------------------